In [18]:
from glob import glob
import pandas as pd
import numpy as np
from copy import deepcopy

In [42]:
dir = '/home/student/Data-analysis-and-presentation/HW1/data/train'
fill_na = True
sepsis_dict = dict()
non_sepsis_dict = dict()

In [43]:
for file in glob(dir + '/patient_*.psv'):
    temp_df = pd.read_csv(file, sep='|')
    # get patient ID
    patient_ID = file[ file.find("_")+1 : file.find(".") ]

    if  temp_df.SepsisLabel.sum() > 0:
        first_sepsis_row = temp_df.shape[0] - temp_df.SepsisLabel.sum()
        sepsis_dict[patient_ID] = dict()
        sepsis_dict[patient_ID]['age'] = temp_df.loc[0, 'Age']
        sepsis_dict[patient_ID]['gender'] = temp_df.loc[0, 'Gender']
        sepsis_dict[patient_ID]['unit1'] = 1 if temp_df.loc[0, 'Unit1'] == 1 else 0
        sepsis_dict[patient_ID]['unit2'] = 1 if temp_df.loc[0, 'Unit2'] == 1 else 0
        sepsis_dict[patient_ID]['unknown unit'] = 0 if temp_df.loc[0, 'Unit1'] == 1 else 0 if temp_df.loc[0, 'Unit2'] == 1 else 1
        sepsis_dict[patient_ID]['HospAdmTime'] = temp_df.loc[0, 'HospAdmTime']

        sepsis_dict[patient_ID]['Final ICULOS'] = temp_df.loc[first_sepsis_row, 'ICULOS']

        sepsis_dict[patient_ID]['Not Null Percentages'] = temp_df.iloc[:first_sepsis_row, :].count() / temp_df.iloc[:first_sepsis_row, :].shape[0]
        sepsis_dict[patient_ID]['Means'] = temp_df.iloc[:first_sepsis_row, :].mean()[:-7]
        sepsis_dict[patient_ID]['Vars'] = temp_df.iloc[:first_sepsis_row, :].var()[:-7]

        if fill_na:
            sepsis_dict[patient_ID]['HR_series'] = temp_df.loc[:first_sepsis_row, 'HR'].interpolate(method='linear').fillna(method='bfill')
            sepsis_dict[patient_ID]['Resp_series'] = temp_df.loc[:first_sepsis_row, 'Resp'].interpolate(method='linear').fillna(method='bfill')
            sepsis_dict[patient_ID]['MAP_series'] = temp_df.loc[:first_sepsis_row, 'MAP'].interpolate(method='linear').fillna(method='bfill')
            sepsis_dict[patient_ID]['O2Sat_series'] = temp_df.loc[:first_sepsis_row, 'O2Sat'].interpolate(method='linear').fillna(method='bfill')
            sepsis_dict[patient_ID]['SBP_series'] = temp_df.loc[:first_sepsis_row, 'SBP'].interpolate(method='linear').fillna(method='bfill')
        else:
            sepsis_dict[patient_ID]['HR_series'] = temp_df.loc[:first_sepsis_row, 'HR']
            sepsis_dict[patient_ID]['Resp_series'] = temp_df.loc[:first_sepsis_row, 'Resp']
            sepsis_dict[patient_ID]['MAP_series'] = temp_df.loc[:first_sepsis_row, 'MAP']
            sepsis_dict[patient_ID]['O2Sat_series'] = temp_df.loc[:first_sepsis_row, 'O2Sat']
            sepsis_dict[patient_ID]['SBP_series'] = temp_df.loc[:first_sepsis_row, 'SBP']
    else:
        non_sepsis_dict[patient_ID] = dict()
        non_sepsis_dict[patient_ID]['age'] = temp_df.loc[0, 'Age']
        non_sepsis_dict[patient_ID]['gender'] = temp_df.loc[0, 'Gender']
        non_sepsis_dict[patient_ID]['unit1'] = 1 if temp_df.loc[0, 'Unit1'] == 1 else 0
        non_sepsis_dict[patient_ID]['unit2'] = 1 if temp_df.loc[0, 'Unit2'] == 1 else 0
        non_sepsis_dict[patient_ID]['unknown unit'] = 0 if temp_df.loc[0, 'Unit1'] == 1 else 0 if temp_df.loc[0, 'Unit2'] == 1 else 1
        non_sepsis_dict[patient_ID]['HospAdmTime'] = temp_df.loc[0, 'HospAdmTime']
        
        non_sepsis_dict[patient_ID]['Final ICULOS'] = temp_df.loc[temp_df.shape[0]-1, 'ICULOS']

        non_sepsis_dict[patient_ID]['Not Null Percentages'] = temp_df.count() / temp_df.shape[0]
        non_sepsis_dict[patient_ID]['Means'] = temp_df.mean()[:-7]
        non_sepsis_dict[patient_ID]['Vars'] = temp_df.var()[:-7]

        if fill_na:
            non_sepsis_dict[patient_ID]['HR_series'] = temp_df['HR'].interpolate(method='linear').fillna(method='bfill')
            non_sepsis_dict[patient_ID]['Resp_series'] = temp_df['Resp'].interpolate(method='linear').fillna(method='bfill')
            non_sepsis_dict[patient_ID]['MAP_series'] = temp_df['MAP'].interpolate(method='linear').fillna(method='bfill')
            non_sepsis_dict[patient_ID]['O2Sat_series'] = temp_df['O2Sat'].interpolate(method='linear').fillna(method='bfill')
            non_sepsis_dict[patient_ID]['SBP_series'] = temp_df['SBP'].interpolate(method='linear').fillna(method='bfill')
        else:
            non_sepsis_dict[patient_ID]['HR_series'] = temp_df['HR']
            non_sepsis_dict[patient_ID]['Resp_series'] = temp_df['Resp']
            non_sepsis_dict[patient_ID]['MAP_series'] = temp_df['MAP']
            non_sepsis_dict[patient_ID]['O2Sat_series'] = temp_df['O2Sat']
            non_sepsis_dict[patient_ID]['SBP_series'] = temp_df['SBP']

In [55]:
sepsis_df = pd.DataFrame.from_dict(data=sepsis_dict, orient='index')
sepsis_df['SepsisLabel'] = 1
non_sepsis_df = pd.DataFrame.from_dict(data=non_sepsis_dict, orient='index')
non_sepsis_df['SepsisLabel'] = 0
all_df = pd.concat([sepsis_df, non_sepsis_df])
all_df.head()

,age,gender,unit1,unit2,unknown unit,HospAdmTime,Final ICULOS,Not Null Percentages,Means,Vars,HR_series,Resp_series,MAP_series,O2Sat_series,SBP_series,SepsisLabel
14421,53.35,1,0,0,1,-0.07,54,HR 0.905660 O2Sat ...,HR 83.906250 O2Sat ...,HR 42.038896 O2Sat ...,0 77.0 1 77.0 2 78.0 3 78....,0 15.000000 1 15.000000 2 14.00000...,0 90.000000 1 90.000000 2 89.00...,0 97.000000 1 97.000000 2 98.00000...,0 123.5 1 123.5 2 119.0 3 108....,1
17558,79.09,1,0,0,1,-0.03,3,HR NaN O2Sat NaN ...,HR NaN O2Sat NaN ...,HR NaN O2Sat NaN ...,"0 79.5 Name: HR, dtype: float64","0 15.5 Name: Resp, dtype: float64","0 112 Name: MAP, dtype: int64","0 100 Name: O2Sat, dtype: int64","0 159 Name: SBP, dtype: int64",1
1873,49.00,0,1,0,0,-70.18,1,HR NaN O2Sat NaN ...,HR NaN O2Sat NaN ...,HR NaN O2Sat NaN ...,"0 NaN Name: HR, dtype: float64","0 NaN Name: Resp, dtype: float64","0 NaN Name: MAP, dtype: float64","0 NaN Name: O2Sat, dtype: float64","0 NaN Name: SBP, dtype: float64",1
14108,55.17,0,0,1,0,-0.02,13,HR 1.000000 O2Sat ...,HR 64.909091 O2Sat ...,HR 21.240909 O2Sat ...,0 70.0 1 69.0 2 69.0 3 69.0 4 ...,0 14.0 1 13.0 2 12.0 3 13.0 4 ...,0 78.0 1 71.0 2 77.0 3 77.0 4 ...,0 95 1 97 2 98 3 98 4 98 5...,0 119.0 1 103.0 2 101.5 3 98....,1
6739,60.00,0,0,1,0,-214.44,69,HR 0.970588 O2Sat ...,HR 109.772727 O2Sat ...,HR 88.732168 O2Sat ...,0 103.5 1 103.5 2 120.0 3 122....,0 27.000000 1 27.000000 2 26.33333...,0 110.0 1 110.0 2 107.0 3 108....,0 90.50 1 90.50 2 90.00 3 94.5...,0 202.0 1 202.0 2 178.5 3 174....,1


In [45]:
all_df.columns
    

Index(['age', 'gender', 'unit1', 'unit2', 'unknown unit', 'HospAdmTime',
       'Final ICULOS', 'Not Null Percentages', 'Means', 'Vars', 'HR_series',
       'Resp_series', 'MAP_series', 'O2Sat_series', 'SBP_series', 'Sepsis'],
      dtype='object')

In [56]:
final_df = deepcopy(all_df[['SepsisLabel','age', 'gender', 'unit1', 'unit2', 'unknown unit', 'HospAdmTime', 'Final ICULOS', 'HR_series', 'Resp_series', 'MAP_series', 'O2Sat_series', 'SBP_series']])

In [57]:
final_df['Temp_var'] = all_df['Vars'].apply(lambda x: x['Temp'])
final_df['Temp_mean'] = all_df['Means'].apply(lambda x: x['Temp'])
final_df['WBC_not_null'] = all_df['Not Null Percentages'].apply(lambda x: x['WBC'])
final_df['WBC_mean'] = all_df['Means'].apply(lambda x: x['WBC'])
final_df['Lactate_not_null'] = all_df['Not Null Percentages'].apply(lambda x: x['Lactate'])
final_df['BaseExcess_not_null'] = all_df['Not Null Percentages'].apply(lambda x: x['BaseExcess'])

In [58]:
final_df.head()

,SepsisLabel,age,gender,unit1,unit2,unknown unit,HospAdmTime,Final ICULOS,HR_series,Resp_series,MAP_series,O2Sat_series,SBP_series,Temp_var,Temp_mean,WBC_not_null,WBC_mean,Lactate_not_null,BaseExcess_not_null
14421,1,53.35,1,0,0,1,-0.07,54,0 77.0 1 77.0 2 78.0 3 78....,0 15.000000 1 15.000000 2 14.00000...,0 90.000000 1 90.000000 2 89.00...,0 97.000000 1 97.000000 2 98.00000...,0 123.5 1 123.5 2 119.0 3 108....,0.318842,37.103333,0.037736,13.3,0.0,0.0
17558,1,79.09,1,0,0,1,-0.03,3,"0 79.5 Name: HR, dtype: float64","0 15.5 Name: Resp, dtype: float64","0 112 Name: MAP, dtype: int64","0 100 Name: O2Sat, dtype: int64","0 159 Name: SBP, dtype: int64",NaN,NaN,NaN,NaN,NaN,NaN
1873,1,49.00,0,1,0,0,-70.18,1,"0 NaN Name: HR, dtype: float64","0 NaN Name: Resp, dtype: float64","0 NaN Name: MAP, dtype: float64","0 NaN Name: O2Sat, dtype: float64","0 NaN Name: SBP, dtype: float64",NaN,NaN,NaN,NaN,NaN,NaN
14108,1,55.17,0,0,1,0,-0.02,13,0 70.0 1 69.0 2 69.0 3 69.0 4 ...,0 14.0 1 13.0 2 12.0 3 13.0 4 ...,0 78.0 1 71.0 2 77.0 3 77.0 4 ...,0 95 1 97 2 98 3 98 4 98 5...,0 119.0 1 103.0 2 101.5 3 98....,0.031633,36.686667,0.090909,11.1,0.0,0.0
6739,1,60.00,0,0,1,0,-214.44,69,0 103.5 1 103.5 2 120.0 3 122....,0 27.000000 1 27.000000 2 26.33333...,0 110.0 1 110.0 2 107.0 3 108....,0 90.50 1 90.50 2 90.00 3 94.5...,0 202.0 1 202.0 2 178.5 3 174....,0.220769,36.292308,0.058824,35.7,0.0,0.0


In [ ]:
final_df.to_pickle('train_data.pkl')